# 評估股價

## 股利法

In [64]:
import pandas as pd
import yfinance as yf

# stock_id = input("請輸入公司股票代號: ")
# stock_id = f"{stock_id}.TW"
maximum_expected_return = int(input("請輸入最大整數預期報酬率: ")) / 100
minumum_expected_return = int(input("請輸入最小整數預期報酬率: ")) / 100
stock_id = "2454.TW"
stock = yf.Ticker(stock_id)
df = pd.DataFrame(stock.dividends)
df = df.reset_index()
df["Dividends"] = round(df["Dividends"], 2)
df["Year"] = df["Date"].dt.year # year 的 type 是 int32
df_dividends = df[df["Year"] == 2024]
dividends = df_dividends["Dividends"].sum().round(2)
expensive_stock_price = dividends / minumum_expected_return
cheap_stock_price = dividends / maximum_expected_return
print("昂貴價:", expensive_stock_price)
print("便宜價:", cheap_stock_price)

昂貴價: 1833.6666666666667
便宜價: 1100.1999999999998


In [29]:
df_dividends

,Date,Dividends,Year
34,2024-03-18 00:00:00+08:00,3.5,2024
35,2024-06-13 00:00:00+08:00,3.5,2024
36,2024-09-12 00:00:00+08:00,4.0,2024
37,2024-12-12 00:00:00+08:00,4.0,2024


## 示警

In [198]:
import time
import json
import requests
import pandas as pd

def get_warning(stock_id, ratio):
    data = {
        "compareItem": ratio,
        "quarter": "true",
        "ylabel": "%",
        "ys": "0",
        "revenue": "true",
        "bcodeAvg": "true",
        "companyAvg": "false",
        "companyId": stock_id
    }

    response = requests.post("https://mopsfin.twse.com.tw/compare/data", data = data)

    data = response.json()
    data = json.loads(data["json"])

    quarters = data["xaxisList"]

    values = data["graphData"]
    full_company_names = data["checkedNameList"]

    df = pd.DataFrame()
    df["Quarters"] = quarters
    company_name_list = ["Quarters"]

    for i in range(len(values)):
        company_data = data["graphData"][i]["data"]
        company_name = data["graphData"][i]["label"]
        company_name_list.append(company_name)
        df2 = pd.DataFrame(company_data)
        df2 = df2.iloc[:, : -1]
        df2 = df2.dropna(axis = 0)
        df2.set_index(0, inplace = True)
        df = pd.concat([df, df2], axis = 1)

    df.columns = company_name_list
    industry_list = full_company_names.copy()
    for i in range(1):
        # print(industry_list[i])
        left_parenthesis_index = industry_list[i].find("(")
        right_parenthesis_index = industry_list[i].find(")")
        industry = industry_list[i][left_parenthesis_index + 3: right_parenthesis_index]
        # industry_list[i] = industry_list[i][left_parenthesis_index + 3: right_parenthesis_index]
        # print(industry)

        company_name = company_name_list[i + 1]
        # print(company_name)

        # print(df[company_name].iloc[-1])
        # print(df[industry].iloc[-1])
        if ratio == "DebtRatio":
            if df[company_name].iloc[-1] > 50:
                if df[company_name].iloc[-1] > df[industry].iloc[-1]:
                    print("Y")
                else:
                    print("N")
            else:
                print("N")
        elif ratio == "LongTermLiabilitiesRatio"or ratio == "CurrentRatio" or ratio == "QuickRatio":
            if df[company_name].iloc[-1] < 100:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    print("Y")
                else:
                    print("N")
            else:
                print("N")
        elif ratio == "InterestCoverage":
            if df[company_name].iloc[-1] < 5:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    print("Y")
                else:
                    print("N")
            else:
                print("N")
        elif ratio == "TotalAssetTurnover":
            if df[company_name].iloc[-1] < 0.5:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    print("Y")
                else:
                    print("N")
            else:
                print("N")
        elif ratio == "NetIncomeMargin" or ratio == "GrossMargin":
            if df[company_name].iloc[-1] < df[company_name].iloc[-5]:
                if ((df[company_name].iloc[-1] - df[company_name].iloc[-5]) / df[company_name].iloc[-5]) < ((df[industry].iloc[-1] - df[industry].iloc[-5]) / df[industry].iloc[-5]):
                    print("Y")
                else:
                    print("N")
            else:
                print("N")
        elif ratio == "ROA":
            if df[company_name].iloc[-1] < 0.06:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    print("Y")
                else:
                    print("N")
            else:
                print("N")
        elif ratio == "ROE":
            if df[company_name].iloc[-1] < 0.08:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    print("Y")
                else:
                    print("N")
            else:
                print("N")  
        elif ratio == "OperatingCashflowToCurrentLiability":
            if df[company_name].iloc[-1] < 1:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    print("Y")
                else:
                    print("N")
            else:
                print("N")       
        elif ratio == "OperatingCashflowToNetProfit":
            if df[company_name].iloc[-1] < 0.8:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    print("Y")
                else:
                    print("N")
            else:
                print("N") 

ratios = ["DebtRatio", "LongTermLiabilitiesRatio", "CurrentRatio", "QuickRatio", "InterestCoverage", "TotalAssetTurnover",
          "GrossMargin", "NetIncomeMargin", "ROA", "ROE", "OperatingCashflowToCurrentLiability", "OperatingCashflowToNetProfit"]

stock_id = input("請輸入公司股票代碼: ")

for i in range(len(ratios)):
    print("ratio:", ratios[i])
    get_warning(stock_id, ratios[i])
    # time.sleep(1)

ratio: DebtRatio
N
ratio: LongTermLiabilitiesRatio
N
ratio: CurrentRatio
N
ratio: QuickRatio
N
ratio: InterestCoverage
N
ratio: TotalAssetTurnover
Y
ratio: GrossMargin
N
ratio: NetIncomeMargin
N
ratio: ROA
N
ratio: ROE
N
ratio: OperatingCashflowToCurrentLiability
N
ratio: OperatingCashflowToNetProfit
N


## 取得十年股價、股利、殖利率

In [410]:
import yfinance as yf

stock_id = "2330.TW"
# stock_id = "2884.TW"
stock = yf.Ticker(stock_id)
# Period must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
stock_data = yf.download(stock_id, period = "max").reset_index()
stock_data

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2000-01-04,69.649033,69.649033,68.475182,69.649033,34.085987,200662336736
1,2000-01-05,69.649033,71.214043,68.866341,71.214043,34.851883,402466805912
2,2000-01-06,70.822884,71.214043,69.649033,69.649033,34.085987,197545715802
3,2000-01-07,67.301331,68.475182,66.518639,67.692490,33.128452,235270344752
4,2000-01-10,69.649033,70.431351,68.475182,70.040192,34.277405,276171685539
...,...,...,...,...,...,...,...
6214,2024-12-13,1060.000000,1070.000000,1055.000000,1065.000000,1065.000000,20423659
6215,2024-12-16,1085.000000,1085.000000,1075.000000,1085.000000,1085.000000,39722392
6216,2024-12-17,1090.000000,1095.000000,1075.000000,1075.000000,1075.000000,37880707
6217,2024-12-18,1075.000000,1090.000000,1070.000000,1085.000000,1085.000000,41863190


In [236]:
stock_data.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object

In [412]:
df_dividends_date = pd.DataFrame(stock.dividends).reset_index()
df_dividends_date = df_dividends_date.drop("Dividends", axis = 1)
df_dividends_date["Date_without_time"] = pd.to_datetime(df_dividends_date["Date"].dt.date) # 原本是文字
df_dividends_date

,Date,Date_without_time
0,2004-06-14 00:00:00+08:00,2004-06-14
1,2005-06-13 00:00:00+08:00,2005-06-13
2,2006-06-20 00:00:00+08:00,2006-06-20
3,2007-06-08 00:00:00+08:00,2007-06-08
4,2008-07-16 00:00:00+08:00,2008-07-16
5,2009-07-15 00:00:00+08:00,2009-07-15
6,2010-07-06 00:00:00+08:00,2010-07-06
7,2011-06-29 00:00:00+08:00,2011-06-29
8,2012-07-04 00:00:00+08:00,2012-07-04
9,2013-07-03 00:00:00+08:00,2013-07-03


In [289]:
df_dividends_date.dtypes

Date                 datetime64[ns, Asia/Taipei]
Date_without_time                 datetime64[ns]
dtype: object

In [413]:
the_date_before_dividends = []

for i in range(len(stock_data["Date"])):
    # print(stock_data["Date"].iloc[i]) 
    if stock_data["Date"].iloc[i] in list(df_dividends_date["Date_without_time"]):
        # print(stock_data["Date"].iloc[i - 1])
        the_date_before_dividends.append(stock_data["Date"].iloc[i - 1])

# the_date_before_dividends
df_dividends_date["the_date_before_dividends"] = the_date_before_dividends
df_dividends_date = df_dividends_date.drop(["Date"], axis = 1) # "Date_without_time", 
df_dividends_date["Year"] = df_dividends_date["the_date_before_dividends"].dt.year
# df_dividends_date["Month"] = df_dividends_date["the_date_before_dividends"].dt.month
# df_dividends_date["Day"] = df_dividends_date["the_date_before_dividends"].dt.day
# df_dividends_date["Week"] = df_dividends_date["the_date_before_dividends"].dt.day_of_week + 1
df_dividends_date

,Date_without_time,the_date_before_dividends,Year
0,2004-06-14,2004-06-11,2004
1,2005-06-13,2005-06-10,2005
2,2006-06-20,2006-06-19,2006
3,2007-06-08,2007-06-07,2007
4,2008-07-16,2008-07-15,2008
5,2009-07-15,2009-07-14,2009
6,2010-07-06,2010-07-05,2010
7,2011-06-29,2011-06-28,2011
8,2012-07-04,2012-07-03,2012
9,2013-07-03,2013-07-02,2013


In [414]:
df_before_dividends_date_and_stock_price = pd.merge(df_dividends_date, stock_data, left_on = "the_date_before_dividends", right_on = "Date")
df_before_dividends_date_and_stock_price = df_before_dividends_date_and_stock_price[["Date_without_time", "the_date_before_dividends", 
                                                                                     "Year", "Close"]]
df_before_dividends_date_and_stock_price = df_before_dividends_date_and_stock_price[df_before_dividends_date_and_stock_price["Year"] >= 2015]
df_before_dividends_date_and_stock_price = df_before_dividends_date_and_stock_price\
    .sort_values("the_date_before_dividends", ascending = False)\
    .reset_index()\
    .drop("index", axis = 1)
df_before_dividends_date_and_stock_price

,Date_without_time,the_date_before_dividends,Year,Close
0,2024-12-12,2024-12-11,2024,1045.0
1,2024-09-12,2024-09-11,2024,901.0
2,2024-06-13,2024-06-12,2024,909.0
3,2024-03-18,2024-03-15,2024,753.0
4,2023-12-14,2023-12-13,2023,577.0
5,2023-09-14,2023-09-13,2023,541.0
6,2023-06-15,2023-06-14,2023,590.0
7,2023-03-16,2023-03-15,2023,511.0
8,2022-12-15,2022-12-14,2022,480.5
9,2022-09-15,2022-09-14,2022,480.0


In [415]:
import requests

stock_id = "2330"
date1 = "104"
date2 = "113"

data = {
    "encodeURIComponent": 1,
    "step": 1,
    "firstin": 1,
    "off": 1,
    "queryName": "co_id",
    "inpuType": "co_id",
    "TYPEK": "all",
    "isnew": "false",
    "co_id": stock_id,
    "date1": date1,
    "date2": date2,
    "qryType": "1"
}
# 你的程式碼中使用的是 requests.get，表示你執行的是 GET 請求，而 data 是用於 POST 的，因此需要改為 params。
response = requests.get("https://mops.twse.com.tw/mops/web/t05st09_2", params = data) 
print(response)

<Response [200]>


In [429]:
import pandas as pd
from bs4 import BeautifulSoup as bs

df_cash_and_stock_dividends = pd.DataFrame()

years_list = list(df_before_dividends_date_and_stock_price["Date_without_time"])[: : -1]
cash_dividends = []
stock_dividends = []

soup = bs(response.text, "html.parser")
# print(soup.prettify())
table = soup.find("table", attrs = {"class": "hasBorder"})
# table 
trs = table.find_all(name = "tr", attrs = {"class": ["odd", "even"]})
for tr in trs:
    tds = tr.find_all(name = "td", attrs = {"align": "right"})
    for i in range(len(tds)):
        if i == 0:
            cash_dividends.append(float(tds[i].text))
        elif i == 4:
            stock_dividends.append(float(tds[i].text))

df_cash_and_stock_dividends["現金股利"] = cash_dividends[: : -1]
df_cash_and_stock_dividends["股票股利"] = stock_dividends[: : -1]
df_cash_and_stock_dividends = df_cash_and_stock_dividends.iloc[: len(years_list)]
# df_cash_and_stock_dividends
df_cash_and_stock_dividends.index = years_list
df_cash_and_stock_dividends = df_cash_and_stock_dividends\
    .reset_index()\
    .rename(columns = {"index": "the_date_after_dividends"})\
    .sort_values("the_date_after_dividends", ascending = False)
ten_years_average_dividends = df_cash_and_stock_dividends["現金股利"].sum() / 10
df_cash_and_stock_dividends

,the_date_after_dividends,現金股利,股票股利
26,2024-12-12,3.999637,0.0
25,2024-09-12,4.000138,0.0
24,2024-06-13,3.499790,0.0
23,2024-03-18,3.499790,0.0
22,2023-12-14,3.000000,0.0
21,2023-09-14,3.000000,0.0
20,2023-06-15,2.749821,0.0
19,2023-03-16,2.749821,0.0
18,2022-12-15,2.750000,0.0
17,2022-09-15,2.750000,0.0


In [427]:
ten_years_average_dividends

9.599887147

In [439]:
df_dividends = pd.merge(df_before_dividends_date_and_stock_price, df_cash_and_stock_dividends, left_on = "Date_without_time", 
                        right_on = "the_date_after_dividends")
df_dividends["殖利率"] = round(df_dividends["現金股利"] / df_dividends["Close"], 4)
df_dividends["現金股利"] = round(df_dividends["現金股利"], 2)
df_dividends

,Date_without_time,the_date_before_dividends,Year,Close,the_date_after_dividends,現金股利,股票股利,殖利率
0,2024-12-12,2024-12-11,2024,1045.0,2024-12-12,4.00,0.0,0.0038
1,2024-09-12,2024-09-11,2024,901.0,2024-09-12,4.00,0.0,0.0044
2,2024-06-13,2024-06-12,2024,909.0,2024-06-13,3.50,0.0,0.0039
3,2024-03-18,2024-03-15,2024,753.0,2024-03-18,3.50,0.0,0.0046
4,2023-12-14,2023-12-13,2023,577.0,2023-12-14,3.00,0.0,0.0052
5,2023-09-14,2023-09-13,2023,541.0,2023-09-14,3.00,0.0,0.0055
6,2023-06-15,2023-06-14,2023,590.0,2023-06-15,2.75,0.0,0.0047
7,2023-03-16,2023-03-15,2023,511.0,2023-03-16,2.75,0.0,0.0054
8,2022-12-15,2022-12-14,2022,480.5,2022-12-15,2.75,0.0,0.0057
9,2022-09-15,2022-09-14,2022,480.0,2022-09-15,2.75,0.0,0.0057


In [438]:
df_dividends["殖利率"].sum() / 10

0.028739999999999998

In [442]:
stock_id = input("請輸入股票代碼: ") + ".TW"
stock_id

'2330.TW'

In [441]:
stock_id = "2330.TW"
dot_index = stock_id.find(".")
stock_id[: dot_index]

'2330'

In [449]:
import requests
import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup as bs

stock_id = input("請輸入股票代碼: ") + ".TW" # "2330.TW"
stock = yf.Ticker(stock_id)
# Period must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
stock_data = yf.download(stock_id, period = "max").reset_index()

df_dividends_date = pd.DataFrame(stock.dividends).reset_index()
df_dividends_date = df_dividends_date.drop("Dividends", axis = 1)
df_dividends_date["Date_without_time"] = pd.to_datetime(df_dividends_date["Date"].dt.date) # 原本是文字

the_date_before_dividends = []

for i in range(len(stock_data["Date"])):
    # print(stock_data["Date"].iloc[i]) 
    if stock_data["Date"].iloc[i] in list(df_dividends_date["Date_without_time"]):
        # print(stock_data["Date"].iloc[i - 1])
        the_date_before_dividends.append(stock_data["Date"].iloc[i - 1])

df_dividends_date["the_date_before_dividends"] = the_date_before_dividends
df_dividends_date = df_dividends_date.drop(["Date"], axis = 1) # "Date_without_time", 
df_dividends_date["Year"] = df_dividends_date["the_date_before_dividends"].dt.year

df_before_dividends_date_and_stock_price = pd.merge(df_dividends_date, stock_data, left_on = "the_date_before_dividends", right_on = "Date")
df_before_dividends_date_and_stock_price = df_before_dividends_date_and_stock_price[["Date_without_time", "the_date_before_dividends", 
                                                                                     "Year", "Close"]]
df_before_dividends_date_and_stock_price = df_before_dividends_date_and_stock_price[df_before_dividends_date_and_stock_price["Year"] >= 2015]
df_before_dividends_date_and_stock_price = df_before_dividends_date_and_stock_price\
    .sort_values("the_date_before_dividends", ascending = False)\
    .reset_index()\
    .drop("index", axis = 1)



dot_index = stock_id.find(".")
date1 = "104"
date2 = "113"

data = {
    "encodeURIComponent": 1,
    "step": 1,
    "firstin": 1,
    "off": 1,
    "queryName": "co_id",
    "inpuType": "co_id",
    "TYPEK": "all",
    "isnew": "false",
    "co_id": stock_id[: dot_index],
    "date1": date1,
    "date2": date2,
    "qryType": "1"
}
# 你的程式碼中使用的是 requests.get，表示你執行的是 GET 請求，而 data 是用於 POST 的，因此需要改為 params。
response = requests.get("https://mops.twse.com.tw/mops/web/t05st09_2", params = data) 

df_cash_and_stock_dividends = pd.DataFrame()

years_list = list(df_before_dividends_date_and_stock_price["Date_without_time"])[: : -1]
cash_dividends = []
stock_dividends = []

soup = bs(response.text, "html.parser")
# print(soup.prettify())
table = soup.find("table", attrs = {"class": "hasBorder"})
# table 
trs = table.find_all(name = "tr", attrs = {"class": ["odd", "even"]})
for tr in trs:
    tds = tr.find_all(name = "td", attrs = {"align": "right"})
    for i in range(len(tds)):
        if i == 0:
            cash_dividends.append(float(tds[i].text))
        elif i == 4:
            stock_dividends.append(float(tds[i].text))

df_cash_and_stock_dividends["現金股利"] = cash_dividends[: : -1]
df_cash_and_stock_dividends["股票股利"] = stock_dividends[: : -1]
df_cash_and_stock_dividends = df_cash_and_stock_dividends.iloc[: len(years_list)]
df_cash_and_stock_dividends.index = years_list
df_cash_and_stock_dividends = df_cash_and_stock_dividends\
    .reset_index()\
    .rename(columns = {"index": "the_date_after_dividends"})\
    .sort_values("the_date_after_dividends", ascending = False)
ten_years_average_dividends = df_cash_and_stock_dividends["現金股利"].sum() / 10

df_dividends = pd.merge(df_before_dividends_date_and_stock_price, df_cash_and_stock_dividends, left_on = "Date_without_time", 
                        right_on = "the_date_after_dividends")
df_dividends["殖利率"] = round(df_dividends["現金股利"] / df_dividends["Close"], 4)
df_dividends["現金股利"] = round(df_dividends["現金股利"], 2)
dividends_yield = df_dividends["殖利率"].sum() / 10

print("近10年平均股利:", ten_years_average_dividends)
print("近10年平均殖利率:", dividends_yield)
df_dividends

[*********************100%%**********************]  1 of 1 completed


近10年平均股利: 5.897
近10年平均殖利率: 0.029819999999999996


,Date_without_time,the_date_before_dividends,Year,Close,the_date_after_dividends,現金股利,股票股利,殖利率
0,2024-06-18,2024-06-17,2024,352.000000,2024-06-18,6.43,0.0,0.0183
1,2023-06-30,2023-06-29,2023,349.500000,2023-06-30,9.84,0.0,0.0282
2,2022-06-29,2022-06-28,2022,236.000000,2022-06-29,5.50,0.0,0.0233
3,2021-07-20,2021-07-19,2021,308.000000,2021-07-20,5.50,0.0,0.0179
4,2020-07-02,2020-07-01,2020,169.500000,2020-07-02,5.00,0.0,0.0295
5,2019-07-02,2019-07-01,2019,160.500000,2019-07-02,5.00,0.0,0.0312
6,2018-07-02,2018-06-29,2018,109.500000,2018-07-02,5.00,0.0,0.0457
7,2017-07-03,2017-06-30,2017,166.500000,2017-07-03,5.00,0.0,0.0300
8,2016-07-07,2016-07-06,2016,158.000000,2016-07-07,5.00,0.0,0.0316
9,2015-06-30,2015-06-29,2015,157.530563,2015-06-30,6.70,0.0,0.0425


## 即時股價、漲跌、漲跌幅

In [166]:
import requests
from bs4 import BeautifulSoup as bs

response = requests.get("https://ws.api.cnyes.com/ws/api/v1/charting/history?resolution=1&symbol=TWS:6214:STOCK&quote=1")
response

<Response [200]>

In [167]:
stock_price = str(response.json()["data"]["quote"]["6"])
stock_price_increase_or_decrease = str(response.json()["data"]["quote"]["220027"])
stock_price_increase_or_decrease_percentage = str(response.json()["data"]["quote"]["56"])
print("股價:", stock_price)
print("股價漲跌:", stock_price_increase_or_decrease)
print("股價漲跌幅:", f"{stock_price_increase_or_decrease_percentage}%")

股價: 139.0
股價漲跌: -0.50
股價漲跌幅: -0.36%


In [474]:
import json
import requests
import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup as bs

stock_id = input("請輸入公司股票代號: ") # 2330
stock_id = f"{stock_id}.TW"
maximum_expected_return = int(input("請輸入最大整數預期報酬率: ")) / 100 # 5
minumum_expected_return = int(input("請輸入最小整數預期報酬率: ")) / 100 # 3
stock = yf.Ticker(stock_id)
df = pd.DataFrame(stock.dividends).reset_index()
# df["Dividends"] = round(df["Dividends"], 2)
df["Year"] = df["Date"].dt.year # year 的 type 是 int32
df_current_year_dividends = df[df["Year"] == 2024]
dividends = df_current_year_dividends["Dividends"].sum()#.round(2)
expensive_stock_price = round(dividends / minumum_expected_return)
cheap_stock_price = round(dividends / maximum_expected_return)
# print("昂貴價:", expensive_stock_price)
# print("便宜價:", cheap_stock_price)

dot_index = stock_id.find(".")

response = requests.get(f"https://ws.api.cnyes.com/ws/api/v1/charting/history?resolution=1&symbol=TWS:{stock_id[: dot_index]}:STOCK&quote=1")

stock_price = str(response.json()["data"]["quote"]["6"])
stock_price_increase_or_decrease = str(response.json()["data"]["quote"]["220027"])
stock_price_increase_or_decrease_percentage = str(response.json()["data"]["quote"]["56"])
# print("股價:", stock_price)
# print("股價漲跌:", stock_price_increase_or_decrease)
# print("股價漲跌幅:", f"{stock_price_increase_or_decrease_percentage}%")

def get_warning(stock_id, ratio):
    data = {
        "compareItem": ratio,
        "quarter": "true",
        "ylabel": "%",
        "ys": "0",
        "revenue": "true",
        "bcodeAvg": "true",
        "companyAvg": "false",
        "companyId": stock_id
    }

    response = requests.post("https://mopsfin.twse.com.tw/compare/data", data = data)

    data = response.json()
    data = json.loads(data["json"])

    quarters = data["xaxisList"]

    values = data["graphData"]
    full_company_names = data["checkedNameList"]

    df = pd.DataFrame()
    df["Quarters"] = quarters
    company_name_list = ["Quarters"]

    for i in range(len(values)):
        company_data = data["graphData"][i]["data"]
        company_name = data["graphData"][i]["label"]
        company_name_list.append(company_name)
        df2 = pd.DataFrame(company_data)
        df2 = df2.iloc[:, : -1]
        df2 = df2.dropna(axis = 0)
        df2.set_index(0, inplace = True)
        df = pd.concat([df, df2], axis = 1)

    df.columns = company_name_list
    industry_list = full_company_names.copy()
    for i in range(1):
        # print(industry_list[i])
        left_parenthesis_index = industry_list[i].find("(")
        right_parenthesis_index = industry_list[i].find(")")
        industry = industry_list[i][left_parenthesis_index + 3: right_parenthesis_index]
        # industry_list[i] = industry_list[i][left_parenthesis_index + 3: right_parenthesis_index]
        # print(industry)

        company_name = company_name_list[i + 1]
        # print(company_name)

        # print(df[company_name].iloc[-1])
        # print(df[industry].iloc[-1])
        if ratio == "DebtRatio":
            if df[company_name].iloc[-1] > 50:
                if df[company_name].iloc[-1] > df[industry].iloc[-1]:
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N")
        elif ratio == "LongTermLiabilitiesRatio"or ratio == "CurrentRatio" or ratio == "QuickRatio":
            if df[company_name].iloc[-1] < 100:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N")
        elif ratio == "InterestCoverage":
            if df[company_name].iloc[-1] < 5:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N")
        elif ratio == "TotalAssetTurnover":
            if df[company_name].iloc[-1] < 0.5:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N")
        elif ratio == "NetIncomeMargin" or ratio == "GrossMargin":
            if df[company_name].iloc[-1] < df[company_name].iloc[-5]:
                if ((df[company_name].iloc[-1] - df[company_name].iloc[-5]) / df[company_name].iloc[-5]) < ((df[industry].iloc[-1] - df[industry].iloc[-5]) / df[industry].iloc[-5]):
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N")
        elif ratio == "ROA":
            if df[company_name].iloc[-1] < 0.06:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N")
        elif ratio == "ROE":
            if df[company_name].iloc[-1] < 0.08:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N") 
        elif ratio == "OperatingCashflowToCurrentLiability":
            if df[company_name].iloc[-1] < 1:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N")      
        elif ratio == "OperatingCashflowToNetProfit":
            if df[company_name].iloc[-1] < 0.8:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N")

ratios = ["DebtRatio", "LongTermLiabilitiesRatio", "CurrentRatio", "QuickRatio", "InterestCoverage", "TotalAssetTurnover",
          "GrossMargin", "NetIncomeMargin", "ROA", "ROE", "OperatingCashflowToCurrentLiability", "OperatingCashflowToNetProfit"]

ratios_values = []

for i in range(len(ratios)):
    # print("ratio:", ratios[i])
    get_warning(stock_id[: dot_index], ratios[i])
    # time.sleep(1)

# Period must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
stock_data = yf.download(stock_id, period = "max").reset_index()

df_dividends_date = df.drop("Dividends", axis = 1)
df_dividends_date["Date_without_time"] = pd.to_datetime(df_dividends_date["Date"].dt.date) # 原本是文字

the_date_before_dividends = []

for i in range(len(stock_data["Date"])):
    # print(stock_data["Date"].iloc[i]) 
    if stock_data["Date"].iloc[i] in list(df_dividends_date["Date_without_time"]):
        # print(stock_data["Date"].iloc[i - 1])
        the_date_before_dividends.append(stock_data["Date"].iloc[i - 1])

df_dividends_date["the_date_before_dividends"] = the_date_before_dividends
df_dividends_date = df_dividends_date.drop(["Date"], axis = 1) # "Date_without_time", 
df_dividends_date["Year"] = df_dividends_date["Date_without_time"].dt.year
df_dividends_date["Quarter"] = df_dividends_date["Date_without_time"].dt.quarter

df_before_dividends_date_and_stock_price = pd.merge(df_dividends_date, stock_data, left_on = "the_date_before_dividends", 
                                                    right_on = "Date")[["Date_without_time", "the_date_before_dividends", 
                                                                        "Year", "Quarter", "Close"]]
df_before_dividends_date_and_stock_price = df_before_dividends_date_and_stock_price[df_before_dividends_date_and_stock_price["Year"] 
                                                                                    >= 2015]
df_before_dividends_date_and_stock_price = df_before_dividends_date_and_stock_price\
    .sort_values("the_date_before_dividends", ascending = False)\
    .reset_index()\
    .drop("index", axis = 1)

beginning_year = "104"
ending_year = "113"

data = {
    "encodeURIComponent": 1,
    "step": 1,
    "firstin": 1,
    "off": 1,
    "queryName": "co_id",
    "inpuType": "co_id",
    "TYPEK": "all",
    "isnew": "false",
    "co_id": stock_id[: dot_index],
    "date1": beginning_year,
    "date2": ending_year,
    "qryType": "1"
}
# 你的程式碼中使用的是 requests.get，表示你執行的是 GET 請求，而 data 是用於 POST 的，因此需要改為 params。
response = requests.get("https://mops.twse.com.tw/mops/web/t05st09_2", params = data) 

df_cash_and_stock_dividends = pd.DataFrame()

years_list = list(df_before_dividends_date_and_stock_price["Date_without_time"])[: : -1]
cash_dividends = []
stock_dividends = []

soup = bs(response.text, "html.parser")
# print(soup.prettify())
table = soup.find("table", attrs = {"class": "hasBorder"})
# table 
trs = table.find_all(name = "tr", attrs = {"class": ["odd", "even"]})
for tr in trs:
    tds = tr.find_all(name = "td", attrs = {"align": "right"})
    for i in range(len(tds)):
        if i == 0:
            cash_dividends.append(float(tds[i].text))
        elif i == 4:
            stock_dividends.append(float(tds[i].text))

df_cash_and_stock_dividends["現金股利"] = cash_dividends[: : -1]
df_cash_and_stock_dividends["股票股利"] = stock_dividends[: : -1]
df_cash_and_stock_dividends = df_cash_and_stock_dividends.iloc[: len(years_list)]
df_cash_and_stock_dividends.index = years_list
df_cash_and_stock_dividends = df_cash_and_stock_dividends\
    .reset_index()\
    .rename(columns = {"index": "the_date_of_dividends"})\
    .sort_values("the_date_of_dividends", ascending = False)
ten_years_average_dividends = df_cash_and_stock_dividends["現金股利"].sum() / 10

df_dividends = pd.merge(df_before_dividends_date_and_stock_price, df_cash_and_stock_dividends, left_on = "Date_without_time", 
                        right_on = "the_date_of_dividends")
df_dividends["殖利率"] = round(df_dividends["現金股利"] / df_dividends["Close"], 4)
df_dividends["現金股利"] = round(df_dividends["現金股利"], 2)
dividends_yield = df_dividends["殖利率"].sum() / 10

# print("近10年平均股利:", ten_years_average_dividends)
# print("近10年平均殖利率:", dividends_yield)
# df_dividends
ratios_values

[*********************100%%**********************]  1 of 1 completed


['N', 'N', 'N', 'N', 'N', 'Y', 'N', 'N', 'N', 'N', 'N', 'N']

## Json 檔

In [ ]:
{
  "stockCode": "2330", 
  "stockPrice": {
    "currentPrice": 500,         
    "priceChange": 10,          
    "priceChangePercent": 2.0,   
    "expensivePrice": 550,       
    "cheapPrice": 450           
  },
  "alerts": {
    "DebtRatio": "Y",              
    "LongTermLiabilitiesRatio": "N",
    "CurrentRatio": "N",
    "QuickRatio": "Y",
    "InterestCoverage": "N",
    "TotalAssetTurnover": "N",
    "GrossMargin": "Y",
    "NetIncomeMargin": "N",
    "ROA": "N",
    "ROE": "Y",
    "OperatingCashflowToCurrentLiability": "N",
    "OperatingCashflowToNetProfit": "Y"
  },
  "dividendHistory": {
    "years": [
      2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015  
    ],
    "last10YearsDividends": [
      10, 12, 14, 16, 15, 13, 11, 10, 9, 8
    ],
    "last10YearsDividendYield": [
      5.0, 5.5, 5.2, 5.1, 4.8, 4.7, 4.9, 5.3, 5.4, 5.6 
    ]
  },
  "averages": {
    "averageDividend": 12.5,            
    "averageDividendYield": 5.1
  }
}

In [ ]:
{
  "stockCode": stock_id, 
  "stockPrice": {
    "currentPrice": stock_price,         
    "priceChange": stock_price_increase_or_decrease,          
    "priceChangePercent": stock_price_increase_or_decrease_percentage,   
    "expensivePrice": expensive_stock_price,       
    "cheapPrice": cheap_stock_price           
  },
  "warning": {
    "DebtRatio": ,              
    "LongTermLiabilitiesRatio": ,
    "CurrentRatio": ,
    "QuickRatio": ,
    "InterestCoverage": ,
    "TotalAssetTurnover": ,
    "GrossMargin": ,
    "NetIncomeMargin": ,
    "ROA": ,
    "ROE": ,
    "OperatingCashflowToCurrentLiability": ,
    "OperatingCashflowToNetProfit": 
  },
  "dividendHistory": {
    "years": [
      2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015  
    ],
    "last10YearsDividends": [
      10, 12, 14, 16, 15, 13, 11, 10, 9, 8
    ],
    "last10YearsDividendYield": [
      5.0, 5.5, 5.2, 5.1, 4.8, 4.7, 4.9, 5.3, 5.4, 5.6 
    ]
  },
  "averages": {
    "averageDividend": ten_years_average_dividends,            
    "averageDividendYield": dividends_yield
  }
}

{'stockCode': '2308.TW',
 'stockPrice': {'currentPrice': '416.5',
  'priceChange': '-9.00',
  'priceChangePercent': '-2.12',
  'expensivePrice': 214,
  'cheapPrice': 129},
 'averages': {'averageDividend': 5.897,
  'averageDividendYield': 0.029819999999999996}}

In [476]:
df_dividends

,Date_without_time,the_date_before_dividends,Year,Quarter,Close,the_date_of_dividends,現金股利,股票股利,殖利率
0,2024-12-12,2024-12-11,2024,4,1045.0,2024-12-12,4.00,0.0,0.0038
1,2024-09-12,2024-09-11,2024,3,901.0,2024-09-12,4.00,0.0,0.0044
2,2024-06-13,2024-06-12,2024,2,909.0,2024-06-13,3.50,0.0,0.0039
3,2024-03-18,2024-03-15,2024,1,753.0,2024-03-18,3.50,0.0,0.0046
4,2023-12-14,2023-12-13,2023,4,577.0,2023-12-14,3.00,0.0,0.0052
5,2023-09-14,2023-09-13,2023,3,541.0,2023-09-14,3.00,0.0,0.0055
6,2023-06-15,2023-06-14,2023,2,590.0,2023-06-15,2.75,0.0,0.0047
7,2023-03-16,2023-03-15,2023,1,511.0,2023-03-16,2.75,0.0,0.0054
8,2022-12-15,2022-12-14,2022,4,480.5,2022-12-15,2.75,0.0,0.0057
9,2022-09-15,2022-09-14,2022,3,480.0,2022-09-15,2.75,0.0,0.0057


In [481]:
for i in range(len(df_dividends)):
    row = df_dividends.iloc[i]
    print(f"{row['Year']}Q{row['Quarter']}", row["現金股利"], row["殖利率"])

2024Q4 4.0 0.0038
2024Q3 4.0 0.0044
2024Q2 3.5 0.0039
2024Q1 3.5 0.0046
2023Q4 3.0 0.0052
2023Q3 3.0 0.0055
2023Q2 2.75 0.0047
2023Q1 2.75 0.0054
2022Q4 2.75 0.0057
2022Q3 2.75 0.0057
2022Q2 2.75 0.0054
2022Q1 2.75 0.0049
2021Q4 2.75 0.0046
2021Q3 2.75 0.0045
2021Q2 2.5 0.0041
2021Q1 2.5 0.0041
2020Q4 2.5 0.0049
2020Q3 2.5 0.0055
2020Q2 2.5 0.0079
2020Q1 2.5 0.0096
2019Q4 2.5 0.0073
2019Q3 2.0 0.0075
2019Q2 8.0 0.0322
2018Q2 8.0 0.0352
2017Q2 7.0 0.0323
2016Q2 6.0 0.0377
2015Q2 4.5 0.0308


In [486]:
dict_format = {
  "stockCode": stock_id, 
  "stockPrice": {
    "currentPrice": stock_price,         
    "priceChange": stock_price_increase_or_decrease,          
    "priceChangePercent": stock_price_increase_or_decrease_percentage,   
    "expensivePrice": expensive_stock_price,       
    "cheapPrice": cheap_stock_price           
  }
}
dict_format["warning"] = {}
for i in range(len(ratios)):
    dict_format["warning"][ratios[i]] = ratios_values[i]

dict_format["dividendHistory"] = {
    "years": [],
    "last10YearsDividends": [],
    "last10YearsDividendYield": []
}

for i in range(len(df_dividends)):
    row = df_dividends.iloc[i]
    # print(f"{row['Year']}Q{row['Quarter']}", row["現金股利"], row["殖利率"])
    dict_format["dividendHistory"]["years"].append(f"{row['Year']}Q{row['Quarter']}")
    dict_format["dividendHistory"]["last10YearsDividends"].append(row["現金股利"])
    dict_format["dividendHistory"]["last10YearsDividendYield"].append(row["殖利率"])

# dict_format["averages"] = {
#    "averageDividend": ten_years_average_dividends,
#    "averageDividendYield": dividends_yield
# }

dict_format

{'stockCode': '2330.TW',
 'stockPrice': {'currentPrice': '1070.0',
  'priceChange': '-15.00',
  'priceChangePercent': '-1.38',
  'expensivePrice': 500,
  'cheapPrice': 300},
 'warning': {'DebtRatio': 'N',
  'LongTermLiabilitiesRatio': 'N',
  'CurrentRatio': 'N',
  'QuickRatio': 'N',
  'InterestCoverage': 'N',
  'TotalAssetTurnover': 'Y',
  'GrossMargin': 'N',
  'NetIncomeMargin': 'N',
  'ROA': 'N',
  'ROE': 'N',
  'OperatingCashflowToCurrentLiability': 'N',
  'OperatingCashflowToNetProfit': 'N'},
 'dividendHistory': {'years': ['2024Q4',
   '2024Q3',
   '2024Q2',
   '2024Q1',
   '2023Q4',
   '2023Q3',
   '2023Q2',
   '2023Q1',
   '2022Q4',
   '2022Q3',
   '2022Q2',
   '2022Q1',
   '2021Q4',
   '2021Q3',
   '2021Q2',
   '2021Q1',
   '2020Q4',
   '2020Q3',
   '2020Q2',
   '2020Q1',
   '2019Q4',
   '2019Q3',
   '2019Q2',
   '2018Q2',
   '2017Q2',
   '2016Q2',
   '2015Q2'],
  'last10YearsDividends': [4.0,
   4.0,
   3.5,
   3.5,
   3.0,
   3.0,
   2.75,
   2.75,
   2.75,
   2.75,
   2.75,
 

In [ ]:
import json
import requests
import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup as bs

stock_id = input("請輸入公司股票代號: ") # 2330
stock_id = f"{stock_id}.TW"
maximum_expected_return = int(input("請輸入最大整數預期報酬率: ")) / 100 # 5
minumum_expected_return = int(input("請輸入最小整數預期報酬率: ")) / 100 # 3
stock = yf.Ticker(stock_id)
df = pd.DataFrame(stock.dividends).reset_index()
# df["Dividends"] = round(df["Dividends"], 2)
df["Year"] = df["Date"].dt.year # year 的 type 是 int32
df_current_year_dividends = df[df["Year"] == 2024]
dividends = df_current_year_dividends["Dividends"].sum()#.round(2)
expensive_stock_price = round(dividends / minumum_expected_return)
cheap_stock_price = round(dividends / maximum_expected_return)
# print("昂貴價:", expensive_stock_price)
# print("便宜價:", cheap_stock_price)

dot_index = stock_id.find(".")

response = requests.get(f"https://ws.api.cnyes.com/ws/api/v1/charting/history?resolution=1&symbol=TWS:{stock_id[: dot_index]}:STOCK&quote=1")

stock_price = str(response.json()["data"]["quote"]["6"])
stock_price_increase_or_decrease = str(response.json()["data"]["quote"]["220027"])
stock_price_increase_or_decrease_percentage = str(response.json()["data"]["quote"]["56"])
# print("股價:", stock_price)
# print("股價漲跌:", stock_price_increase_or_decrease)
# print("股價漲跌幅:", f"{stock_price_increase_or_decrease_percentage}%")

def get_warning(stock_id, ratio):
    data = {
        "compareItem": ratio,
        "quarter": "true",
        "ylabel": "%",
        "ys": "0",
        "revenue": "true",
        "bcodeAvg": "true",
        "companyAvg": "false",
        "companyId": stock_id
    }

    response = requests.post("https://mopsfin.twse.com.tw/compare/data", data = data)

    data = response.json()
    data = json.loads(data["json"])

    quarters = data["xaxisList"]

    values = data["graphData"]
    full_company_names = data["checkedNameList"]

    df = pd.DataFrame()
    df["Quarters"] = quarters
    company_name_list = ["Quarters"]

    for i in range(len(values)):
        company_data = data["graphData"][i]["data"]
        company_name = data["graphData"][i]["label"]
        company_name_list.append(company_name)
        df2 = pd.DataFrame(company_data)
        df2 = df2.iloc[:, : -1]
        df2 = df2.dropna(axis = 0)
        df2.set_index(0, inplace = True)
        df = pd.concat([df, df2], axis = 1)

    df.columns = company_name_list
    industry_list = full_company_names.copy()
    for i in range(1):
        # print(industry_list[i])
        left_parenthesis_index = industry_list[i].find("(")
        right_parenthesis_index = industry_list[i].find(")")
        industry = industry_list[i][left_parenthesis_index + 3: right_parenthesis_index]
        # industry_list[i] = industry_list[i][left_parenthesis_index + 3: right_parenthesis_index]
        # print(industry)

        company_name = company_name_list[i + 1]
        # print(company_name)

        # print(df[company_name].iloc[-1])
        # print(df[industry].iloc[-1])
        if ratio == "DebtRatio":
            if df[company_name].iloc[-1] > 50:
                if df[company_name].iloc[-1] > df[industry].iloc[-1]:
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N")
        elif ratio == "LongTermLiabilitiesRatio"or ratio == "CurrentRatio" or ratio == "QuickRatio":
            if df[company_name].iloc[-1] < 100:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N")
        elif ratio == "InterestCoverage":
            if df[company_name].iloc[-1] < 5:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N")
        elif ratio == "TotalAssetTurnover":
            if df[company_name].iloc[-1] < 0.5:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N")
        elif ratio == "NetIncomeMargin" or ratio == "GrossMargin":
            if df[company_name].iloc[-1] < df[company_name].iloc[-5]:
                if ((df[company_name].iloc[-1] - df[company_name].iloc[-5]) / df[company_name].iloc[-5]) < ((df[industry].iloc[-1] - df[industry].iloc[-5]) / df[industry].iloc[-5]):
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N")
        elif ratio == "ROA":
            if df[company_name].iloc[-1] < 0.06:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N")
        elif ratio == "ROE":
            if df[company_name].iloc[-1] < 0.08:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N") 
        elif ratio == "OperatingCashflowToCurrentLiability":
            if df[company_name].iloc[-1] < 1:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N")      
        elif ratio == "OperatingCashflowToNetProfit":
            if df[company_name].iloc[-1] < 0.8:
                if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                    ratios_values.append("Y")
                else:
                    ratios_values.append("N")
            else:
                ratios_values.append("N")

ratios = ["DebtRatio", "LongTermLiabilitiesRatio", "CurrentRatio", "QuickRatio", "InterestCoverage", "TotalAssetTurnover",
          "GrossMargin", "NetIncomeMargin", "ROA", "ROE", "OperatingCashflowToCurrentLiability", "OperatingCashflowToNetProfit"]

ratios_values = []

for i in range(len(ratios)):
    # print("ratio:", ratios[i])
    get_warning(stock_id[: dot_index], ratios[i])
    # time.sleep(1)

# Period must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
stock_data = yf.download(stock_id, period = "max").reset_index()

df_dividends_date = df.drop("Dividends", axis = 1)
df_dividends_date["Date_without_time"] = pd.to_datetime(df_dividends_date["Date"].dt.date) # 原本是文字

the_date_before_dividends = []

for i in range(len(stock_data["Date"])):
    # print(stock_data["Date"].iloc[i]) 
    if stock_data["Date"].iloc[i] in list(df_dividends_date["Date_without_time"]):
        # print(stock_data["Date"].iloc[i - 1])
        the_date_before_dividends.append(stock_data["Date"].iloc[i - 1])

df_dividends_date["the_date_before_dividends"] = the_date_before_dividends
df_dividends_date = df_dividends_date.drop(["Date"], axis = 1) # "Date_without_time", 
df_dividends_date["Year"] = df_dividends_date["Date_without_time"].dt.year
df_dividends_date["Quarter"] = df_dividends_date["Date_without_time"].dt.quarter

df_before_dividends_date_and_stock_price = pd.merge(df_dividends_date, stock_data, left_on = "the_date_before_dividends", 
                                                    right_on = "Date")[["Date_without_time", "the_date_before_dividends", 
                                                                        "Year", "Quarter", "Close"]]
df_before_dividends_date_and_stock_price = df_before_dividends_date_and_stock_price[df_before_dividends_date_and_stock_price["Year"] 
                                                                                    >= 2015]
df_before_dividends_date_and_stock_price = df_before_dividends_date_and_stock_price\
    .sort_values("the_date_before_dividends", ascending = False)\
    .reset_index()\
    .drop("index", axis = 1)

beginning_year = "104"
ending_year = "113"

data = {
    "encodeURIComponent": 1,
    "step": 1,
    "firstin": 1,
    "off": 1,
    "queryName": "co_id",
    "inpuType": "co_id",
    "TYPEK": "all",
    "isnew": "false",
    "co_id": stock_id[: dot_index],
    "date1": beginning_year,
    "date2": ending_year,
    "qryType": "1"
}
# 你的程式碼中使用的是 requests.get，表示你執行的是 GET 請求，而 data 是用於 POST 的，因此需要改為 params。
response = requests.get("https://mops.twse.com.tw/mops/web/t05st09_2", params = data) 

df_cash_and_stock_dividends = pd.DataFrame()

years_list = list(df_before_dividends_date_and_stock_price["Date_without_time"])[: : -1]
cash_dividends = []
stock_dividends = []

soup = bs(response.text, "html.parser")
# print(soup.prettify())
table = soup.find("table", attrs = {"class": "hasBorder"})
# table 
trs = table.find_all(name = "tr", attrs = {"class": ["odd", "even"]})
for tr in trs:
    tds = tr.find_all(name = "td", attrs = {"align": "right"})
    for i in range(len(tds)):
        if i == 0:
            cash_dividends.append(float(tds[i].text))
        elif i == 4:
            stock_dividends.append(float(tds[i].text))

df_cash_and_stock_dividends["現金股利"] = cash_dividends[: : -1]
df_cash_and_stock_dividends["股票股利"] = stock_dividends[: : -1]
df_cash_and_stock_dividends = df_cash_and_stock_dividends.iloc[: len(years_list)]
df_cash_and_stock_dividends.index = years_list
df_cash_and_stock_dividends = df_cash_and_stock_dividends\
    .reset_index()\
    .rename(columns = {"index": "the_date_of_dividends"})\
    .sort_values("the_date_of_dividends", ascending = False)
ten_years_average_dividends = round(df_cash_and_stock_dividends["現金股利"].sum() / 10, 2)

df_dividends = pd.merge(df_before_dividends_date_and_stock_price, df_cash_and_stock_dividends, left_on = "Date_without_time", 
                        right_on = "the_date_of_dividends")
df_dividends["殖利率"] = round(df_dividends["現金股利"] / df_dividends["Close"], 4)
df_dividends["現金股利"] = round(df_dividends["現金股利"], 2)
dividends_yield = round(df_dividends["殖利率"].sum() / 10, 2)

# print("近10年平均股利:", ten_years_average_dividends)
# print("近10年平均殖利率:", dividends_yield)
# df_dividends
# ratios_values

dict_format = {
  "stockCode": stock_id, 
  "stockPrice": {
    "currentPrice": stock_price,         
    "priceChange": stock_price_increase_or_decrease,          
    "priceChangePercent": stock_price_increase_or_decrease_percentage,   
    "expensivePrice": expensive_stock_price,       
    "cheapPrice": cheap_stock_price           
  }
}
dict_format["warning"] = {}
for i in range(len(ratios)):
    dict_format["warning"][ratios[i]] = ratios_values[i]

dict_format["dividendHistory"] = {
    "years": [],
    "last10YearsDividends": [],
    "last10YearsDividendYield": []
}

for i in range(len(df_dividends)):
    row = df_dividends.iloc[i]
    # print(f"{row['Year']}Q{row['Quarter']}", row["現金股利"], row["殖利率"])
    dict_format["dividendHistory"]["years"].append(f"{row['Year']}Q{row['Quarter']}")
    dict_format["dividendHistory"]["last10YearsDividends"].append(row["現金股利"])
    dict_format["dividendHistory"]["last10YearsDividendYield"].append(row["殖利率"])

dict_format["averages"] = {
   "averageDividend": ten_years_average_dividends,
   "averageDividendYield": dividends_yield
}

dict_format

[*********************100%%**********************]  1 of 1 completed


{'stockCode': '3037.TW',
 'stockPrice': {'currentPrice': '142.5',
  'priceChange': '1.00',
  'priceChangePercent': '0.71',
  'expensivePrice': 100,
  'cheapPrice': 60},
 'warning': {'DebtRatio': 'Y',
  'LongTermLiabilitiesRatio': 'N',
  'CurrentRatio': 'N',
  'QuickRatio': 'N',
  'InterestCoverage': 'N',
  'TotalAssetTurnover': 'Y',
  'GrossMargin': 'N',
  'NetIncomeMargin': 'N',
  'ROA': 'N',
  'ROE': 'N',
  'OperatingCashflowToCurrentLiability': 'Y',
  'OperatingCashflowToNetProfit': 'Y'},
 'dividendHistory': {'years': ['2024Q3',
   '2023Q3',
   '2022Q3',
   '2021Q3',
   '2020Q3',
   '2019Q3',
   '2018Q3',
   '2017Q3',
   '2016Q3',
   '2015Q3'],
  'last10YearsDividends': [3.0, 8.0, 3.4, 1.4, 1.1, 0.8, 0.5, 0.3, 0.18, 0.29],
  'last10YearsDividendYield': [0.0171,
   0.0468,
   0.0221,
   0.01,
   0.0211,
   0.0194,
   0.0299,
   0.0155,
   0.0125,
   0.0187]},
 'averages': {'averageDividend': 1.8965,
  'averageDividendYield': 0.021310000000000003}}

In [490]:
with open("3037_stock_price.json", "w", encoding = "utf-8") as file:
    json.dump(dict_format, fp = file, ensure_ascii = False, indent = 2)

In [493]:
import json
import requests
import pandas as pd
import yfinance as yf
from flask import Flask, request
from bs4 import BeautifulSoup as bs

app = Flask(__name__)

@app.route("/get_stock_price", methods = ["get"])
def stock_price():
    stock_id = input("請輸入公司股票代號: ") # 2330
    stock_id = f"{stock_id}.TW"
    maximum_expected_return = int(input("請輸入最大整數預期報酬率: ")) / 100 # 5
    minumum_expected_return = int(input("請輸入最小整數預期報酬率: ")) / 100 # 3
    stock = yf.Ticker(stock_id)
    df = pd.DataFrame(stock.dividends).reset_index()
    # df["Dividends"] = round(df["Dividends"], 2)
    df["Year"] = df["Date"].dt.year # year 的 type 是 int32
    df_current_year_dividends = df[df["Year"] == 2024]
    dividends = df_current_year_dividends["Dividends"].sum()#.round(2)
    expensive_stock_price = round(dividends / minumum_expected_return)
    cheap_stock_price = round(dividends / maximum_expected_return)
    # print("昂貴價:", expensive_stock_price)
    # print("便宜價:", cheap_stock_price)

    dot_index = stock_id.find(".")

    response = requests.get(f"https://ws.api.cnyes.com/ws/api/v1/charting/history?resolution=1&symbol=TWS:{stock_id[: dot_index]}:STOCK&quote=1")

    stock_price = str(response.json()["data"]["quote"]["6"])
    stock_price_increase_or_decrease = str(response.json()["data"]["quote"]["220027"])
    stock_price_increase_or_decrease_percentage = str(response.json()["data"]["quote"]["56"])
    # print("股價:", stock_price)
    # print("股價漲跌:", stock_price_increase_or_decrease)
    # print("股價漲跌幅:", f"{stock_price_increase_or_decrease_percentage}%")

    def get_warning(stock_id, ratio):
        data = {
            "compareItem": ratio,
            "quarter": "true",
            "ylabel": "%",
            "ys": "0",
            "revenue": "true",
            "bcodeAvg": "true",
            "companyAvg": "false",
            "companyId": stock_id
        }

        response = requests.post("https://mopsfin.twse.com.tw/compare/data", data = data)

        data = response.json()
        data = json.loads(data["json"])

        quarters = data["xaxisList"]

        values = data["graphData"]
        full_company_names = data["checkedNameList"]

        df = pd.DataFrame()
        df["Quarters"] = quarters
        company_name_list = ["Quarters"]

        for i in range(len(values)):
            company_data = data["graphData"][i]["data"]
            company_name = data["graphData"][i]["label"]
            company_name_list.append(company_name)
            df2 = pd.DataFrame(company_data)
            df2 = df2.iloc[:, : -1]
            df2 = df2.dropna(axis = 0)
            df2.set_index(0, inplace = True)
            df = pd.concat([df, df2], axis = 1)

        df.columns = company_name_list
        industry_list = full_company_names.copy()
        for i in range(1):
            # print(industry_list[i])
            left_parenthesis_index = industry_list[i].find("(")
            right_parenthesis_index = industry_list[i].find(")")
            industry = industry_list[i][left_parenthesis_index + 3: right_parenthesis_index]
            # industry_list[i] = industry_list[i][left_parenthesis_index + 3: right_parenthesis_index]
            # print(industry)

            company_name = company_name_list[i + 1]
            # print(company_name)

            # print(df[company_name].iloc[-1])
            # print(df[industry].iloc[-1])
            if ratio == "DebtRatio":
                if df[company_name].iloc[-1] > 50:
                    if df[company_name].iloc[-1] > df[industry].iloc[-1]:
                        ratios_values.append("Y")
                    else:
                        ratios_values.append("N")
                else:
                    ratios_values.append("N")
            elif ratio == "LongTermLiabilitiesRatio"or ratio == "CurrentRatio" or ratio == "QuickRatio":
                if df[company_name].iloc[-1] < 100:
                    if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                        ratios_values.append("Y")
                    else:
                        ratios_values.append("N")
                else:
                    ratios_values.append("N")
            elif ratio == "InterestCoverage":
                if df[company_name].iloc[-1] < 5:
                    if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                        ratios_values.append("Y")
                    else:
                        ratios_values.append("N")
                else:
                    ratios_values.append("N")
            elif ratio == "TotalAssetTurnover":
                if df[company_name].iloc[-1] < 0.5:
                    if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                        ratios_values.append("Y")
                    else:
                        ratios_values.append("N")
                else:
                    ratios_values.append("N")
            elif ratio == "NetIncomeMargin" or ratio == "GrossMargin":
                if df[company_name].iloc[-1] < df[company_name].iloc[-5]:
                    if ((df[company_name].iloc[-1] - df[company_name].iloc[-5]) / df[company_name].iloc[-5]) < ((df[industry].iloc[-1] - df[industry].iloc[-5]) / df[industry].iloc[-5]):
                        ratios_values.append("Y")
                    else:
                        ratios_values.append("N")
                else:
                    ratios_values.append("N")
            elif ratio == "ROA":
                if df[company_name].iloc[-1] < 0.06:
                    if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                        ratios_values.append("Y")
                    else:
                        ratios_values.append("N")
                else:
                    ratios_values.append("N")
            elif ratio == "ROE":
                if df[company_name].iloc[-1] < 0.08:
                    if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                        ratios_values.append("Y")
                    else:
                        ratios_values.append("N")
                else:
                    ratios_values.append("N") 
            elif ratio == "OperatingCashflowToCurrentLiability":
                if df[company_name].iloc[-1] < 1:
                    if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                        ratios_values.append("Y")
                    else:
                        ratios_values.append("N")
                else:
                    ratios_values.append("N")      
            elif ratio == "OperatingCashflowToNetProfit":
                if df[company_name].iloc[-1] < 0.8:
                    if df[company_name].iloc[-1] < df[industry].iloc[-1]:
                        ratios_values.append("Y")
                    else:
                        ratios_values.append("N")
                else:
                    ratios_values.append("N")

    ratios = ["DebtRatio", "LongTermLiabilitiesRatio", "CurrentRatio", "QuickRatio", "InterestCoverage", "TotalAssetTurnover",
            "GrossMargin", "NetIncomeMargin", "ROA", "ROE", "OperatingCashflowToCurrentLiability", "OperatingCashflowToNetProfit"]

    ratios_values = []

    for i in range(len(ratios)):
        # print("ratio:", ratios[i])
        get_warning(stock_id[: dot_index], ratios[i])
        # time.sleep(1)

    # Period must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
    stock_data = yf.download(stock_id, period = "max").reset_index()

    df_dividends_date = df.drop("Dividends", axis = 1)
    df_dividends_date["Date_without_time"] = pd.to_datetime(df_dividends_date["Date"].dt.date) # 原本是文字

    the_date_before_dividends = []

    for i in range(len(stock_data["Date"])):
        # print(stock_data["Date"].iloc[i]) 
        if stock_data["Date"].iloc[i] in list(df_dividends_date["Date_without_time"]):
            # print(stock_data["Date"].iloc[i - 1])
            the_date_before_dividends.append(stock_data["Date"].iloc[i - 1])

    df_dividends_date["the_date_before_dividends"] = the_date_before_dividends
    df_dividends_date = df_dividends_date.drop(["Date"], axis = 1) # "Date_without_time", 
    df_dividends_date["Year"] = df_dividends_date["Date_without_time"].dt.year
    df_dividends_date["Quarter"] = df_dividends_date["Date_without_time"].dt.quarter

    df_before_dividends_date_and_stock_price = pd.merge(df_dividends_date, stock_data, left_on = "the_date_before_dividends", 
                                                        right_on = "Date")[["Date_without_time", "the_date_before_dividends", 
                                                                            "Year", "Quarter", "Close"]]
    df_before_dividends_date_and_stock_price = df_before_dividends_date_and_stock_price[df_before_dividends_date_and_stock_price["Year"] 
                                                                                        >= 2015]
    df_before_dividends_date_and_stock_price = df_before_dividends_date_and_stock_price\
        .sort_values("the_date_before_dividends", ascending = False)\
        .reset_index()\
        .drop("index", axis = 1)

    beginning_year = "104"
    ending_year = "113"

    data = {
        "encodeURIComponent": 1,
        "step": 1,
        "firstin": 1,
        "off": 1,
        "queryName": "co_id",
        "inpuType": "co_id",
        "TYPEK": "all",
        "isnew": "false",
        "co_id": stock_id[: dot_index],
        "date1": beginning_year,
        "date2": ending_year,
        "qryType": "1"
    }
    # 你的程式碼中使用的是 requests.get，表示你執行的是 GET 請求，而 data 是用於 POST 的，因此需要改為 params。
    response = requests.get("https://mops.twse.com.tw/mops/web/t05st09_2", params = data) 

    df_cash_and_stock_dividends = pd.DataFrame()

    years_list = list(df_before_dividends_date_and_stock_price["Date_without_time"])[: : -1]
    cash_dividends = []
    stock_dividends = []

    soup = bs(response.text, "html.parser")
    # print(soup.prettify())
    table = soup.find("table", attrs = {"class": "hasBorder"})
    # table 
    trs = table.find_all(name = "tr", attrs = {"class": ["odd", "even"]})
    for tr in trs:
        tds = tr.find_all(name = "td", attrs = {"align": "right"})
        for i in range(len(tds)):
            if i == 0:
                cash_dividends.append(float(tds[i].text))
            elif i == 4:
                stock_dividends.append(float(tds[i].text))

    df_cash_and_stock_dividends["現金股利"] = cash_dividends[: : -1]
    df_cash_and_stock_dividends["股票股利"] = stock_dividends[: : -1]
    df_cash_and_stock_dividends = df_cash_and_stock_dividends.iloc[: len(years_list)]
    df_cash_and_stock_dividends.index = years_list
    df_cash_and_stock_dividends = df_cash_and_stock_dividends\
        .reset_index()\
        .rename(columns = {"index": "the_date_of_dividends"})\
        .sort_values("the_date_of_dividends", ascending = False)
    ten_years_average_dividends = round(df_cash_and_stock_dividends["現金股利"].sum() / 10, 2)

    df_dividends = pd.merge(df_before_dividends_date_and_stock_price, df_cash_and_stock_dividends, left_on = "Date_without_time", 
                            right_on = "the_date_of_dividends")
    df_dividends["殖利率"] = round(df_dividends["現金股利"] / df_dividends["Close"], 4)
    df_dividends["現金股利"] = round(df_dividends["現金股利"], 2)
    dividends_yield = round(df_dividends["殖利率"].sum() / 10, 4)

    # print("近10年平均股利:", ten_years_average_dividends)
    # print("近10年平均殖利率:", dividends_yield)
    # df_dividends
    # ratios_values

    dict_format = {
    "stockCode": stock_id, 
    "stockPrice": {
        "currentPrice": stock_price,         
        "priceChange": stock_price_increase_or_decrease,          
        "priceChangePercent": stock_price_increase_or_decrease_percentage,   
        "expensivePrice": expensive_stock_price,       
        "cheapPrice": cheap_stock_price           
    }
    }
    dict_format["warning"] = {}
    for i in range(len(ratios)):
        dict_format["warning"][ratios[i]] = ratios_values[i]

    dict_format["dividendHistory"] = {
        "years": [],
        "last10YearsDividends": [],
        "last10YearsDividendYield": []
    }

    for i in range(len(df_dividends)):
        row = df_dividends.iloc[i]
        # print(f"{row['Year']}Q{row['Quarter']}", row["現金股利"], row["殖利率"])
        dict_format["dividendHistory"]["years"].append(f"{row['Year']}Q{row['Quarter']}")
        dict_format["dividendHistory"]["last10YearsDividends"].append(row["現金股利"])
        dict_format["dividendHistory"]["last10YearsDividendYield"].append(row["殖利率"])

    dict_format["averages"] = {
    "averageDividend": ten_years_average_dividends,
    "averageDividendYield": dividends_yield
    }

    return json.dumps(dict_format, ensure_ascii = False, indent = 2)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/Dec/2024 10:53:11] "GET / HTTP/1.1" 404 -
[*********************100%%**********************]  1 of 1 completed
127.0.0.1 - - [20/Dec/2024 10:53:34] "GET /get_stock_price HTTP/1.1" 200 -


In [507]:
import http.client

conn = http.client.HTTPConnection("127.0.0.1", 5000)  # 改用 HTTP
payload = ''
headers = {}
conn.request("GET", "/get_stock_price?stock_id=2330&maximum_expected_return=5&minumum_expected_return=3", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

{
  "stockCode": "2330.TW",
  "stockPrice": {
    "currentPrice": "1050.0",
    "priceChange": "-20.00",
    "priceChangePercent": "-1.87",
    "expensivePrice": 500,
    "cheapPrice": 300
  },
  "warning": {
    "DebtRatio": "N",
    "LongTermLiabilitiesRatio": "N",
    "CurrentRatio": "N",
    "QuickRatio": "N",
    "InterestCoverage": "N",
    "TotalAssetTurnover": "Y",
    "GrossMargin": "N",
    "NetIncomeMargin": "N",
    "ROA": "N",
    "ROE": "N",
    "OperatingCashflowToCurrentLiability": "N",
    "OperatingCashflowToNetProfit": "N"
  },
  "dividendHistory": {
    "years": [
      "2024Q4",
      "2024Q3",
      "2024Q2",
      "2024Q1",
      "2023Q4",
      "2023Q3",
      "2023Q2",
      "2023Q1",
      "2022Q4",
      "2022Q3",
      "2022Q2",
      "2022Q1",
      "2021Q4",
      "2021Q3",
      "2021Q2",
      "2021Q1",
      "2020Q4",
      "2020Q3",
      "2020Q2",
      "2020Q1",
      "2019Q4",
      "2019Q3",
      "2019Q2",
      "2018Q2",
      "2017Q2",
      "20

In [516]:
temp = {
  "stockCode": "2330.TW",
  "stockPrice": {
    "currentPrice": "1050.0",
    "priceChange": "-20.00",
    "priceChangePercent": "-1.87",
    "expensivePrice": 500,
    "cheapPrice": 300
  },
  "warning": {
    "DebtRatio": "N",
    "LongTermLiabilitiesRatio": "N",
    "CurrentRatio": "N",
    "QuickRatio": "N",
    "InterestCoverage": "N",
    "TotalAssetTurnover": "Y",
    "GrossMargin": "N",
    "NetIncomeMargin": "N",
    "ROA": "N",
    "ROE": "N",
    "OperatingCashflowToCurrentLiability": "N",
    "OperatingCashflowToNetProfit": "N"
  },
  "dividendHistory": {
    "years": [
      "2024Q4",
      "2024Q3",
      "2024Q2",
      "2024Q1",
      "2023Q4",
      "2023Q3",
      "2023Q2",
      "2023Q1",
      "2022Q4",
      "2022Q3",
      "2022Q2",
      "2022Q1",
      "2021Q4",
      "2021Q3",
      "2021Q2",
      "2021Q1",
      "2020Q4",
      "2020Q3",
      "2020Q2",
      "2020Q1",
      "2019Q4",
      "2019Q3",
      "2019Q2",
      "2018Q2",
      "2017Q2",
      "2016Q2",
      "2015Q2"
    ],
    "last10YearsDividends": [
      4.0,
      4.0,
      3.5,
      3.5,
      3.0,
      3.0,
      2.75,
      2.75,
      2.75,
      2.75,
      2.75,
      2.75,
      2.75,
      2.75,
      2.5,
      2.5,
      2.5,
      2.5,
      2.5,
      2.5,
      2.5,
      2.0,
      8.0,
      8.0,
      7.0,
      6.0,
      4.5
    ],
    "last10YearsDividendYield": [
      0.0038,
      0.0045,
      0.0039,
      0.0047,
      0.0053,
      0.0057,
      0.0048,
      0.0056,
      0.006,
      0.006,
      0.0057,
      0.0052,
      0.0049,
      0.0049,
      0.0044,
      0.0044,
      0.0053,
      0.006,
      0.0087,
      0.0107,
      0.0081,
      0.0084,
      0.0375,
      0.0424,
      0.0402,
      0.0489,
      0.0412
    ]
  },
  "averages": {
    "averageDividend": 9.6,
    "averageDividendYield": 0.0337
  }
}

In [517]:
with open("2330_stock_price.json", "w", encoding = "utf-8") as file:
    json.dump(temp, fp = file, ensure_ascii = False, indent = 2)

## ROE 法

In [ ]:
import requests

stock_id = input("請輸入一間公司股票代碼: ")

data = {
    "compareItem": "ROE",
    "quarter": "false",
    "ylabel": "%",
    "ys": "0",
    "revenue": "true",
    "bcodeAvg": "true",
    "companyAvg": "false",
    "qnumber": 4,
    "companyId": stock_id
}

response = requests.post("https://mopsfin.twse.com.tw/compare/data", data = data)
response

In [ ]:
response.json()

In [ ]:
import pandas as pd

years_list = response.json()["xaxisList"][-5: ]
industry_data = response.json()["graphData"][-1]["data"][: -1][-5: ]
industry_name = response.json()["graphData"][-1]["label"]
for i in range(len(years_list)):
    years_list[i] = years_list[i].replace("Q4", "")

df_ROE_industry_average = pd.DataFrame(industry_data, index = years_list)
df_ROE_industry_average = df_ROE_industry_average.drop([0, 2], axis = 1)
df_ROE_industry_average = df_ROE_industry_average.rename(columns = {1: "ROE"})
# df_ROE_industry_average
ROE_industry_average = df_ROE_industry_average["ROE"].mean()
ROE_industry_average.round(2)